In [6]:
!pip install tinydb

In [9]:
import sqlite3
from tinydb import TinyDB, Query
import json

# Step 1: Connect to SQLite and TinyDB
conn = sqlite3.connect('db1.sqlite')
cursor = conn.cursor()

# Initialize TinyDB
tinydb = TinyDB('db1_schema.json')
schema_table = tinydb.table('schema_versions')

# Step 2: Extract the current schema from SQLite
def extract_schema():
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    schema_metadata = {}

    for table in tables:
        table_name = table[0]
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        schema_metadata[table_name] = [{"column_id": col[0], "name": col[1], "type": col[2], "notnull": col[3], "default": col[4], "primary_key": col[5]} for col in columns]

    return schema_metadata

# Step 3: Version the schema and store it in TinyDB
def store_schema_version(schema, version):
    # Create a schema entry with a version number
    schema_entry = {
        "version": version,
        "schema": schema
    }
    # Insert into TinyDB
    schema_table.insert(schema_entry)

# Step 4: Retrieve the current schema version from TinyDB
def get_latest_version():
    if len(schema_table) == 0:
        return 0  # No version exists, start from 0
    else:
        return max([entry['version'] for entry in schema_table.all()])

# Step 5: Main workflow to extract, version, and store the schema
def version_and_store_schema():
    # Extract current schema from SQLite
    schema = extract_schema()

    # Get the latest version from TinyDB
    current_version = get_latest_version()

    # Increment version for new schema
    new_version = current_version + 1

    # Store the schema with the new version in TinyDB
    store_schema_version(schema, new_version)

    print(f"Schema version {new_version} stored successfully.")

# Step 6: Run the versioning process
version_and_store_schema()

# Step 7: Verify stored schema versions (Optional)
def verify_schema_versions():
    all_versions = schema_table.all()
    for entry in all_versions:
        print(f"Version {entry['version']} Schema:")
        print(json.dumps(entry['schema'], indent=4))

verify_schema_versions()

# Close the connections
conn.close()
tinydb.close()


Schema version 2 stored successfully.
Version 1 Schema:
{
    "Tenant": [
        {
            "column_id": 0,
            "name": "tenant_id",
            "type": "INTEGER",
            "notnull": 0,
            "default": null,
            "primary_key": 1
        },
        {
            "column_id": 1,
            "name": "name",
            "type": "TEXT",
            "notnull": 1,
            "default": null,
            "primary_key": 0
        }
    ],
    "sqlite_sequence": [
        {
            "column_id": 0,
            "name": "name",
            "type": "",
            "notnull": 0,
            "default": null,
            "primary_key": 0
        },
        {
            "column_id": 1,
            "name": "seq",
            "type": "",
            "notnull": 0,
            "default": null,
            "primary_key": 0
        }
    ],
    "Category": [
        {
            "column_id": 0,
            "name": "id",
            "type": "INTEGER",
            "notnull"